# This will be the demo integration of Colivara with OpenRouter

#### Import the required libraries


In [ ]:
# utilities
import base64 # for converting docs binaries to base64
from pathlib import Path 
import getpass
import os 
import requests 

from colivara_py import ColiVara 

In [ ]:
# Setup Environment Variables
os.environ['OPEN_ROUTER_API_KEY'] = getpass.getpass()
os.environ['COLIVARA_API_KEY'] = getpass.getpass()


#### Download example docs for our demo (you can use your own docs as well)

In [ ]:
def download_file(url, local_filename):
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Ensure the 'docs' directory exists
        os.makedirs("docs", exist_ok=True)

        # Write the content to a local file
        with open(local_filename, "wb") as f:
            f.write(response.content)
        print(f"Successfully downloaded: {local_filename}")
    else:
        print(f"Failed to download: {url}")


# URLs and local filenames
files = [
    {
        "url": "https://github.com/tjmlabs/colivara-demo/raw/main/docs/Work-From-Home%20Guidance.pdf",
        "filename": "docs/Work-From-Home-Guidance.pdf",
    },
    {
        "url": "https://github.com/tjmlabs/colivara-demo/raw/main/docs/StaffVendorPolicy-Jan2019.pdf",
        "filename": "docs/StaffVendorPolicy-Jan2019.pdf",
    },
]

In [ ]:
# Download each file
for file in files:
    download_file(file["url"], file["filename"])


#### Start Colivara Client and Create Collection `to upsert documents into`.

In [ ]:
rag_client = ColiVara(
    base_url="https://api.colivara.com", api_key=os.environ["COLIVARA_API_KEY"]
)

In [ ]:
rag_client.create_collection("openrouter-demo")

#### Upsert the documents to the collection

In [ ]:
def sync_documents():
    # get all the documents under docs/ folder and upsert them to colivara
    documents_dir = Path("docs")
    files = list(documents_dir.glob("**/*"))

    for file in files:
        with open(file, "rb") as f:
            file_content = f.read()
            encoded_content = base64.b64encode(file_content).decode("utf-8")
            rag_client.upsert_document(
                name=file.name,
                document_base64=encoded_content,
                collection_name="openrouter-demo",
                wait=True,
            )
            print(f"Upserted: {file.name}")

In [ ]:
sync_documents()

There is a step called query transformation, which is the process of converting the user query into a format that the RAG model can understand. 
This step is necessary because the RAG model expects the input to be in a specific format.

***for the purpose of the demo, we will skip it and use the RAG query as the input to the RAG directly.***

Setup ended successfully.
Let's begin work.


In [ ]:
query = "What is the work from home policy?"

you can do search by just using this code snippet:

```python
rag_client.search(query="some query",collection_name="collection_name", top_k=5)
``` 

In [ ]:
## you can try performing search to see results object structure

# results = rag_client.search(query, collection_name="openrouter-demo")
# print(results.results)

In [ ]:
# get context using the RAG
def get_context(query):
    results = rag_client.search(query=query, collection_name="openrouter-demo", top_k=3)
    results = results.results

    context = []
    for result in results:
        base64 = result.img_base64
        # base64 doesn't have data: part so we need to add it
        if "data:image" not in base64:
            base64 = f"data:image/png;base64,{base64}"
        context.append(base64)
    return context

In [ ]:
context = get_context(query)

#### Now prepare the request. Then, generate the response.

In [ ]:
url = "https://openrouter.ai/api/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {os.environ['OPEN_ROUTER_API_KEY']}",
    "Content-Type": "application/json",
}


payload = {
    "model": "openai/gpt-4o",
    "messages": [
        {
            "role": "system",
            "content": "You are a helpful assistant that answers questions based on the provided images/docs.",
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Here are some images for context:",
                },
                *[
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": path,
                        },
                    }
                    for path in context
                ],
                {"type": "text", "text": query},
            ],
        },
    ],
    "max_tokens": 500,
    "temperature": 0.0,
}

In [ ]:
response = requests.post(url, headers=headers, json=payload)

In [ ]:
print(response.json()["choices"][0]["message"]["content"])

## That's it for now. You can continue to explore more features and capabilities of different LLMs.